In [1]:
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable

# O método da posição falsa

## $ \S 1 $ Descrição do método da posição falsa

Suponha que a função real contínua $ f \colon [a, b] \to \mathbb R $ seja tal que
$$
    \operatorname{sinal} f(a) \ne \operatorname{sinal} f(b)\,.
$$
Assim como o método da bissecção, o __método da posição falsa__ começa com as
duas estimativas $ a $ e $ b $ para um zero. Mas em vez de tomar a próxima
estimativa como o ponto médio destes, construímos a reta pelos pontos
$$
    \big(a, f(a)\big) \quad \text{e} \quad \big(b,f(b)\big)
$$
e encontramos o ponto onde ela cruza o eixo-$x$, definindo $ c $ como sua
primeira coordenada. Este seria exatamente o zero de $ f $ caso ela fosse linear
entre $ a $ e $ b $, mas em geral isto não acontecerá. Portanto há três
possibilidades:
* Se $ f(c) = 0 $, então $ c $ é um zero de $ f $ e podemos terminar.
* Se $ \operatorname{sinal} f(c) \neq \operatorname{sinal} f(a) $, então $ f $
  troca de sinal em $ [a, c] $. Neste caso fazemos $ b \leftarrow c $.
* Se $ \operatorname{sinal} f(c) = \operatorname{sinal} f(a) $, então $ f $
  troca de sinal em $ [c, b] $. Neste caso fazemos $ a \leftarrow c $.

Nos dois últimos casos repetimos o procedimento usando o novo intervalo $ [a, b]
$ em lugar do original e assim sucessivamente, até que seja satisfeito o
critério de parada. Observe que em cada iteração temos a garantia da existência
de um zero dentro do intervalo sob consideração, pois $ f $ troca de sinal aí.

📝 O método da posição falsa também é conhecido como método _regula falsi_.

__Exemplo 1:__ Execute em seqüência as três células abaixo para ver uma animação
do método da posição falsa aproximando o zero da função $ f(x) = 1 + x\cos x + \sin(2x) $
em $ 1.8394 $ (aproximadamente).

In [2]:
def print_solution(xs: list[float], ys: list[float], freq: int = 1) -> None:
    """
    Given two arrays xs and ys of the same length, prints a table whose n-th
    line consists of three entries: the values of n, xs[n] and ys[n].
    Parameters:
        * The arrays xs and ys. 
        * A parameter freq used to print only one in every freq line. The
          first, second and last line are always printed. If freq == 0, then
          only these lines are printed.
    Output: None.
    Prints: A header and the table described above.
    """
    def print_header() -> None:
        """
        Prints the table's header.
        """
        print("\n|       n      ", end="")
        print("    x_n            ", end="")
        print("    f(x_n)      |")
        print("|=================================================|")
        
    def print_line(n: int, x: float, y: float) -> None:
        """
        Pretty-prints n, x and y.
        """
        if n == 0:
            print(f"|       a", end="")
        elif n == 1:
            print(f"|       b", end="")
        else:
            print(f"|      {n - 1:02}", end="")
        print(f"    {x:15.8f}", end="")
        print(f"    {y:15.8f}   |")
    
    
    N = len(xs)
    if freq == 0:       # If freq == 0, print only first and last lines.
        freq = N - 1
    print_header()
    for n in range(0, 2):
        print_line(n, xs[n], ys[n])
    for n in range(2, N, freq):
        print_line(n, xs[n], ys[n])
    if n != N - 1:
        print_line(n, xs[N - 1], ys[N - 1])
    print("|_________________________________________________|\n")
        
    return None

In [25]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from typing import Callable, List, Tuple

def false_position_animation(f: Callable[[float], float],
                             a: float,
                             b: float,
                             N: int = 4,
                             title: str = "",
                             duration: float = 0.75
                             ) -> tuple[list[float], list[float], HTML]:
    """
    Displays an animation of the false position (a.k.a. regula falsi) method
    applied to a function.

    Parameters:
    * f: A continuous real function f.
    * a: The left endpoint of the interval [a, b] such that f(a)f(b) < 0.
    * b: The right endpoint of the interval [a, b] such that f(a)f(b) < 0.
    * N: The maximum number of iterations.
    * title: A title to be displayed at the top of the diagram.
    * duration: The duration of the pause between slides of the animation, in seconds.

    Returns:
    * xs: List of float, the estimates from the false position method.
    * ys: List of float, the values of the function f at each of the estimates.
    * animation_html: IPython.display.HTML object containing the animation.
    """

    def iterate(a, b):
        """Applies a single step of the false position method to the interval [a, b]."""
        c = (a * f(b) - b * f(a)) / (f(b) - f(a)) 
        if np.sign(f(a)) != np.sign(f(c)):  # [a, c] contains a zero
            return c, a, c
        else:  # [c, b] contains a zero
            return c, c, b

    cmap = plt.get_cmap("tab10")
    P = 200
    width = 1.75
    marker_size = 5
    domain = np.linspace(a, b, P)
    xs = [a, b]
    ys = [f(a), f(b)]
    xs_line = [np.linspace(a, b, P)]
    ys_line = [np.linspace(f(a), f(b), P)]
    for _ in range(N):
        c, a, b = iterate(a, b)
        xs.append(c)
        xs_line.append(np.linspace(a, b, P))
        ys_line.append(np.linspace(f(a), f(b), P))
    ys = [f(x) for x in xs]
    xs_vert = [np.linspace(xs[n], xs[n], P) for n in range(N + 2)]
    ys_vert = [np.linspace(0, ys[n], P) for n in range(N + 2)]

    fig, ax = plt.subplots()
    ax.axhline(y=0.0, color='black', linestyle='-', lw=width)
    ax.plot(domain, f(domain), label="$ y = f(x) $", lw=width)
    ax.set_xlabel("$ x $-axis")
    ax.set_ylabel("$ y $-axis")
    ax.set_title(title)
    ax.grid(True)

    lines = []
    for i in range(N + 2):
        color = cmap(i + 1)
        label = None
        if i == 0:
            label = "$ a $"
        elif i == 1:
            label = "$ b $"
        else:
            label = f"$ x_{i - 1} $"
        (line,) = ax.plot([], [], color=color, marker="x", mew=width, label=label)
        (vert_line,) = ax.plot([], [], linestyle='dotted', lw=width, color=color)
        (point,) = ax.plot([], [], color='black', marker="o", ms=marker_size)
        (sec_line,) = ax.plot([], [], linestyle='--', lw=width, color=color)
        lines.append((line, vert_line, point, sec_line))

    def init():
        for line, vert_line, point, sec_line in lines:
            line.set_data([], [])
            vert_line.set_data([], [])
            point.set_data([], [])
            sec_line.set_data([], [])
        return [item for sublist in lines for item in sublist]

    def update(frame):
        if frame == 0:
            return [item for sublist in lines for item in sublist]
        elif frame == (N + 2) * 2:
            ax.legend()  # Show legend at the end
            return [item for sublist in lines for item in sublist]
        n = (frame - 1) // 2
        substep = (frame - 1) % 2
        if n < N + 2:
            x = xs[n]
            y = ys[n]
            line, vert_line, point, sec_line = lines[n]
            if substep == 0:
                line.set_data([x], [0])
                vert_line.set_data(xs_vert[n], ys_vert[n])
                point.set_data([x], [y])
            else:
                if n > 0:
                    sec_line.set_data(xs_line[n - 1], ys_line[n - 1])
        return [item for sublist in lines for item in sublist]

    ani = FuncAnimation(fig, update, frames=range((N + 2) * 2 + 1), init_func=init,
                        blit=True, interval=duration * 500)
    
    plt.close(fig)
    return xs, ys, HTML(ani.to_jshtml())

In [26]:
from numpy import sin, pi, cos, log, exp
a = 0.0           # Extremidade esquerda do intervalo inicial, onde f vale -3.
b = pi            # Extremidade direita, onde f vale 3.
N = 4             # Número de iterações desejado.
pausa = 1.0      # Intervalo entre cada passo da animação, em segundos.
# Função à qual o método será aplicado:
f = lambda x: 1 + x * cos(x) + sin(2 * x)

# Título a ser exibido no topo do diagrama:
titulo = "Método da posição falsa para "\
         "$ y = 1 + x\\cos x + \\sin(2x), a = 0, b = \\pi $"

xs, ys, anim = false_position_animation(f, a, b, N, titulo, pausa)
anim

![Exemplo do método da posição falsa](fig_2-4_exemplo_1.png "Exemplo de aplicação do método da posição falsa")

In [10]:
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|       a         0.00000000         1.00000000   |
|       b         3.14159265        -2.14159265   |
|      01         1.00000000         2.44959973   |
|      02         2.14263232        -1.06970379   |
|      03         1.79532549         0.16615807   |
|      04         1.84201990        -0.00972982   |
|_________________________________________________|



## $ \S 3 $ Fórmula para a próxima estimativa no método da falsa posição

A reta por dois pontos $ (x_0, y_0) $ e $ (x_1, y_1) $ é caracterizada pela
igualdade entre a inclinação do segmento que liga $ (x_1, y_1) $ a
$ (x_0, y_0) $ e a do segmento ligando um ponto $ (x, y) $ qualquer sobre a reta
a $ (x_0, y_0) $. Em símbolos:
$$
    \frac{y - y_0}{x - x_0} = \frac{y_1 - y_0}{x_1 - x_0}
$$
Remanejando, obtemos a equação equivalente
$$
    y = y_0 + \frac{y_1 - y_0}{x_1 - x_0}\,(x - x_0).
$$

No método da posição falsa utilizamos em cada passo a reta passando por
$ \big(a,f(a)\big) $ e $ \big(b,f(b)\big) $, onde $ a $ e $ b $ são as
extremidades do intervalo obtido no passo anterior. Substituindo estes valores
acima, obtemos a equação
$$
    y = f(a) + \frac{f(b) - f(a)}{b - a}\,(x - a).
$$
A próxima estimativa $ c $ para o zero é o único valor de $ x $ que faz esta expressão se anular:
$$
    \boxed{c = \frac{af(b) - bf(a)}{f(b)-f(a)}}
$$ 

📝 Observe a simetria desta fórmula com respeito a $ a $ e $ b $ e o fato que o
denominador é não-nulo pois $ f(a) $ e $ f(b) $ têm sinais opostos por hipótese.

📝 No método da bissecção, a próxima estimativa é a média _aritmética_ de $ a $
e $ b $. Contudo, em geral é de se esperar que se $ \vert f(a) \vert $ seja
menor que $ \vert f(b) \vert $, então o zero $ \zeta $ que está sendo aproximado
esteja mais perto de $ a $ que de $ b $. Isto sugere a que tomemos a próxima
estimativa $ c $ como a média de $ a $ e $ b $ _ponderada_ por $ \vert f(a)
\vert $ e $ \vert f(b) \vert $. O resultado é justamente a fórmula do método da
posição falsa, já que, como $ f(a) $ e $ f(b) $ têm sinais opostos por hipótese,
vale
$$
  c = \frac{a \vert f(b) \vert - b \vert f(a) \vert}
  {\vert f(b) \vert - \vert f(a)\vert }
   = \frac{af(b) - bf(a)}{f(b)-f(a)}
$$


__Problema 1:__ Para cada uma das funções abaixo, observe que $ \zeta = 1 $ é um
zero. Sem usar o computador, calcule duas iterações do método da falsa posição
para aproximá-lo, utilizando o intervalo inicial indicado.

(a) $ f(x) = x^2 - 4x + 3 $ no intervalo $ [0, 3] $.

(b) $ g(x) = x^3 - 3x^2 - x + 3 $ no intervalo $ [0, 2] $.

_Solução:_

## $ \S 4 $ Análise informal do erro no método da posição falsa

Suponha por concretude que o sinal de $ f(c) $ seja o mesmo que o de $ f(a) $,
de modo que o intervalo seguinte seja $ [c, b] $. Podemos estimar o novo erro
$ b - c $ em termos do anterior $ b - a $:
\begin{alignat*}{3}
    b - c &= \frac{bf(b) - {bf(a)} -\big[af(b) - {bf(a)}\big]}{f(b) - f(a)} \\
    & = \frac{f(b)}{f(b) - f(a)} (b - a)\,.
\end{alignat*}
Como por hipótese $ f(a) $ e $ f(b) $ têm sinais opostos, o fator que multiplica
$ (b - a) $ está entre $ 0 $ e $ 1 $. Se $ f(b) $ for muito maior que $ f(a) $
em valor absoluto, este fator ficará próximo de $ 1 $. Se isto acontecer em
todas as iterações, a convergência ao zero será lenta; esta situação é ilustrada
pela animação abaixo.

In [28]:
a = 0          # Extremidade esquerda do intervalo inicial, onde f vale -3.
b = 4          # Extremidade direita, onde f vale 57.
N = 7          # Número de iterações desejado.
pausa = 0.5    # Intervalo de tempo entre cada passo da animação, em segundos.
f = lambda x: x**3 - x - 3    # Função à qual o método será aplicado.
# Título a ser exibido no topo do diagrama:
titulo = ("Método da posição falsa para $ y = x^3 - x - 3, a = 0, b = 4 $."
          "\nAproximação lenta por um único lado.")

xs, ys, ani = false_position_animation(f, a, b, N, titulo, pausa)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|       a         0.00000000        -3.00000000   |
|       b         4.00000000        57.00000000   |
|      01         0.20000000        -3.19200000   |
|      02         0.40151515        -3.33678512   |
|      03         0.60052096        -3.38395783   |
|      04         0.79103006        -3.29605996   |
|      05         0.96644712        -3.06376615   |
|      06         1.12118428        -2.71179488   |
|      07         1.25192491        -2.28976299   |
|_________________________________________________|



![Exemplo de convergência lenta com o método da posição falsa](fig_2-4_exemplo_2.png "Exemplo de convergência lenta com o método da posição falsa")

Não é possível calcular precisamente e de maneira geral o erro cometido pelo
método da posição falsa. Porém na situação descrita acima, seu desempenho é
consideravelmente pior que o do método do bissecção. Por este motivo ele
raramente é empregado na prática.

📝 Uma outra desvantagem do método da posição falsa é que, apesar de ser
possível mostrar que a seqüência $ (x_n) $ das estimativas geradas por ele
sempre converge a um zero, nem sempre o comprimento do intervalo $ [a_n, b_n] $
correspondente tende a $ 0 $ conforme $ n \to \infty $; isto é
evidenciado no exemplo estudado acima.


📝 Para exemplos ainda mais extremos da lentidão da convergência, troque o expoente $ n = 3 $ por outro maior na expressão para a função $ f(x) = x^n - x - 3 $ considerada acima e rode novamente a animação.

## $ \S 3 $ Implementação do método da posição falsa

Como o método da bissecção e da posição falsa só diferem na fórmula usada para obter a nova estimativa para o zero, as duas implementações são muito parecidas.

In [31]:
from typing import Callable, List, Tuple
import numpy as np

def false_position(f: Callable[[float], float],
                   a: float,
                   b: float,
                   eps: float = 1e-3,
                   max_iter: int = 100
                   ) -> tuple[list[float], list[float]]:
    """
    Uses the false position method to approximate a zero of a function.

    * f: Real continuous function.
    * a, b: Points such that f(a)f(b) < 0.
    * eps: Maximum tolerance for the error.
    * max_iter: Maximum number of iterations.

    Returns:
        * Two lists, xs and ys, with estimates and values of f at each estimate.
    """
    xs = [a, b]
    ys = [f(a), f(b)]
    iterations = 0

    while (b - a) >= eps and iterations < max_iter:
        c = (a * f(b) - b * f(a)) / (f(b) - f(a))
        xs.append(c)
        ys.append(f(c))

        if f(c) == 0:
            return xs, ys
        elif np.sign(f(a)) != np.sign(f(c)):
            b = c
        else:
            a = c
        iterations += 1
        
    return xs, ys

## $ \S 4 $ Problemas

__Problema 2:__ 

(a) Sem usar o computador, aplique quatro iterações do método da posição falsa à
função $ f(x) = x^2 - 2 $ posição para obter uma aproximação racional
$ \sqrt{2} $ a partir do intervalo inicial $ [1, 2] $.

(b) Usando Python, calcule o erro absoluto e o erro relativo resultantes.

_Solução:_

__Problema 3:__ Modifique a implementação do método da posição falsa de modo que
o procedimento só termine caso:
* O número máximo de iterações seja atingido; ou
* $ \vert f(x_n) \vert < \delta $, onde a tolerância
  $ \delta > 0 $ é passada como argumento no lugar de $ \varepsilon $.

_Solução:_

__Problema 4:__ Usando o método da falsa posição com precisão de três dígitos decimais:

(a) Encontre uma raiz positiva da equação $ \sin x = \frac{x}{2} $.

(a) Encontre a primeira raiz positiva da equação $ \sin x = \frac{x}{n} $ para $ n = 2, 3, \dots, 20 $.

*Solução:*

__Problema 5:__ Calcule com ajuda do computador, mas sem usar a implementação
acima, as três primeiras iterações do método da posição falsa para estimar uma
raiz das equações abaixo nos intervalos indicados. Esboce também os gráficos das
funções utilizadas:

(a) $ \tan x - \frac{1}{1 + x^2} = 0 $, $ 0 \le x \le \frac{\pi}{2} $.

(b) $ x^2 = 2 $, $ 0 \le x \le 2 $.

(c) $ x \ln x = 1 $, $ 1 \le x \le 2 $.

(d) $ \cos x = x $, $ 0 \le x \le \frac{\pi}{2} $.

*Solução:*

__Problema 6:__ Vimos no caderno anterior que no método da bissecção os
comprimentos do intervalo anterior e do atual estão relacionados pela fórmula 
$$
\left\vert{I_n}\right\vert = c \left\vert{I_{n-1}}\right\vert \quad \text{com} \quad c = \frac{1}{2}
$$ 

(a) Estime a constante $ c $ que relaciona as duas para o método da falsa
posição aplicado ao problema de se encontrar a raiz positiva da equação
$$
x^{8} - 1 = 0
$$
usando como intervalo inicial $ [a, b] = [0, 2] $.

(b) Discuta se seria melhor usar o método da bissecção ou o método da falsa posição neste caso. 

_Solução:_

__Problema 7:__ Seja $ g(x) = e^x - x - 2 $.

(a) Mostre que existe um _único_ zero de $ g $ em $ [-2, 0] $.

(b) Utilizando o método da bissecção, quantas iterações seriam necessárias (a princípio) para aproximar
este zero com precisão melhor que $ \varepsilon = 10^{-7} $?

(c) Usando nossa implementação do método da posição falsa, aproxime este zero
usando no máximo $ 10 $ iterações com precisão desejada de $ \varepsilon $.
Esta precisão é atingida?

(d) Utilize a animação do método da posição falsa ou o gráfico de $ g $ para
explicar o resultado obtido em (c).

_Solução:_